<a href="https://colab.research.google.com/github/JoyKarmoker/Python-Codes-ML/blob/master/Gradient_descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# Load the Drive helper and mount
from google.colab import drive
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras import layers

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/drive/MyDrive/insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age', 'affordibility']], df.bought_insurance, test_size = 0.2, random_state=25)

In [5]:
X_train

,age,affordibility
0,22,1
13,29,0
6,55,0
17,58,1
24,50,1
19,18,1
25,54,1
16,25,0
20,21,1
3,52,0


In [6]:
len(X_train)

22

In [7]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age']/100

In [8]:
X_train_scaled

,age,affordibility
0,0.22,1
13,0.29,0
6,0.55,0
17,0.58,1
24,0.50,1
19,0.18,1
25,0.54,1
16,0.25,0
20,0.21,1
3,0.52,0


In [9]:
X_test_scaled

,age,affordibility
2,0.47,1
10,0.18,1
21,0.26,0
11,0.28,1
14,0.49,1
9,0.61,1


In [ ]:
model = keras.Sequential([
    layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X_train_scaled, y_train, epochs=5000)

In [11]:
model.evaluate(X_test_scaled, y_test)

1/1 [==============================] - 0s 169ms/step - loss: 0.3550 - accuracy: 1.0000


[0.35497748851776123, 1.0]

In [12]:
X_test_scaled

,age,affordibility
2,0.47,1
10,0.18,1
21,0.26,0
11,0.28,1
14,0.49,1
9,0.61,1


In [13]:
model.predict(X_test_scaled)

array([[0.7054848 ],
       [0.35569546],
       [0.16827849],
       [0.47801176],
       [0.7260697 ],
       [0.8294984 ]], dtype=float32)

In [14]:
coef, intercept = model.get_weights()
coef, intercept

(array([[5.060868],
        [1.40865 ]], dtype=float32), array([-2.913703], dtype=float32))

In [15]:
def sigmoid(x):
  import math
  return 1/(1+math.exp(-x))
sigmoid(18)

0.9999999847700205

In [16]:
def prediction_function(age, affordibility):
  weighted_sum = coef[0]*age + coef[1]*affordibility + intercept[0]
  return sigmoid(weighted_sum)

In [17]:
prediction_function(0.47, 1)

0.705484819775958

In [18]:
prediction_function(0.18, 1)

0.35569544317951163

In [36]:

def log_loss(y_true, y_predicted):
  epsilon = 1e-15
  y_prdicted_new = [max(i, epsilon) for i in y_predicted]
  y_predicted_new = [min(i, 1-epsilon) for i in y_predicted_new]
  y_predicted_new = np.array(y_predicted_new)
  return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [30]:
def sigmoid_numpy(x):
  return 1/(1+np.exp(-x))

In [32]:
sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [33]:
def gradiant_descent(age, affordibility, y_true, epochs, loss_threshold):
  w1=w2=1
  bias = 0
  rate = 0.5
  n = len(age)
  for i in range(epochs):
    weighted_sum = w1*age + w2*affordibility + bias
    y_predicted = sigmoid_numpy(weighted_sum)

    loss = log_loss(y_true, y_predicted)
    
    w1d = (1/n) * np.dot(np.transpose(age), y_predicted-y_true)
    w2d = (1/n) * np.dot(np.transpose(affordibility), y_predicted-y_true)
    bias_d = np.mean(y_predicted - y_true)

    w1 = w1-w1*w1d
    w2 = w2-w2*w2d
    bias = bias - bias_d

    print(f'EPOCH: {i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

    if loss<=loss_threshold:
      break
  
  return w1, w2, bias


In [40]:
gradiant_descent(X_train_scaled['age'], X_train_scaled['affordibility'], y_train, 1000, 0.4631)

EPOCH: 0, w1:0.9498152669403539, w2:0.8966962507890579, bias:-0.22683735472737165, loss:0.7113403233723417
EPOCH: 1, w1:0.925054634713377, w2:0.8387009996567901, bias:-0.3932380739944269, loss:0.6559178222899531
EPOCH: 2, w1:0.9179139705490423, w2:0.8095107320319233, bias:-0.513913643538202, loss:0.6286684873489866
EPOCH: 3, w1:0.9226761521745337, w2:0.7983600986264011, bias:-0.6024477911489164, loss:0.6153132049481876
EPOCH: 4, w1:0.9354941951727498, w2:0.7985505060503829, bias:-0.6692140353250584, loss:0.6083147723024861
EPOCH: 5, w1:0.9538489989846476, w2:0.8059113665963158, bias:-0.7215087281993792, loss:0.6041239934316331
EPOCH: 6, w1:0.9760948106684387, w2:0.8177962505475932, bias:-0.764278637573615, loss:0.6011559821572502
EPOCH: 7, w1:1.0011510512265671, w2:0.8324892641996668, bias:-0.8008187132064427, loss:0.5987206464625551
EPOCH: 8, w1:1.0283049158126472, w2:0.8488579438805842, bias:-0.8333016093246198, loss:0.5965217939255623
EPOCH: 9, w1:1.0570854661066305, w2:0.8661455799

(5.2994718388834645, 1.2356293673076815, -2.8398890739602574)

In [41]:
coef, intercept

(array([[5.060868],
        [1.40865 ]], dtype=float32), array([-2.913703], dtype=float32))